# 📈 TVS Motor Stock Price Prediction

This notebook demonstrates how to use **Linear Regression** (and later Polynomial Regression) to predict the stock price of **TVS Motor (TVSMOTOR.NS)** using Yahoo Finance data.

In [8]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.1 MB 2.6 MB/s eta 0:00:01
  Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
     |████████████████████████████████| 30.3 MB 5.9 MB/s eta 0:00:011
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'matplotlib'

## Step 1: Download TVS Motor stock data

In [11]:
ticker = yf.Ticker("TVSMOTOR.NS")
data = ticker.history(start="2022-01-01", end="2025-08-01")

if data.empty:
    raise ValueError("No data downloaded. Please check ticker or internet.")

data.head(60)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00+05:30,613.124758,620.923717,604.590049,617.294006,1188070,0.00,0.0
2022-01-04 00:00:00+05:30,619.010790,623.474327,614.596292,618.471252,574936,0.00,0.0
2022-01-05 00:00:00+05:30,620.776471,631.959889,617.441109,626.564392,1056096,0.00,0.0
2022-01-06 00:00:00+05:30,622.934700,637.649693,618.029703,636.570618,1042775,0.00,0.0
2022-01-07 00:00:00+05:30,638.483535,641.573707,625.141965,628.869751,715951,0.00,0.0
2022-01-10 00:00:00+05:30,632.744705,640.592702,630.292207,637.453491,771849,0.00,0.0
2022-01-11 00:00:00+05:30,634.706751,640.396537,628.967927,630.488464,389969,0.00,0.0
2022-01-12 00:00:00+05:30,632.744736,645.644905,630.292238,642.358521,810304,0.00,0.0
2022-01-13 00:00:00+05:30,644.958185,650.451759,630.782729,647.901184,1892618,0.00,0.0


## Step 2: Feature Engineering

In [12]:
data["Lag1"] = data["Close"].shift(1)
data["MA5"] = data["Close"].rolling(5).mean()
data["VolumeLag1"] = data["Volume"].shift(1)
data = data.dropna()

data.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Lag1,MA5,VolumeLag1
Date,,,,,,,,,,
2025-07-25 00:00:00+05:30,2797.699951,2799.899902,2728.699951,2774.399902,909208,0.0,0.0,2797.699951,2801.119971,395856.0
2025-07-28 00:00:00+05:30,2774.399902,2832.800049,2750.000000,2798.899902,601547,0.0,0.0,2774.399902,2793.799951,909208.0
2025-07-29 00:00:00+05:30,2790.000000,2825.000000,2784.899902,2801.800049,820647,0.0,0.0,2798.899902,2795.079980,601547.0
2025-07-30 00:00:00+05:30,2814.899902,2825.000000,2754.500000,2793.600098,624411,0.0,0.0,2801.800049,2793.279980,820647.0
2025-07-31 00:00:00+05:30,2760.000000,2846.000000,2755.000000,2801.800049,1287444,0.0,0.0,2793.600098,2794.100000,624411.0


## Step 3: Train-Test Split

In [13]:
X = data[["Lag1", "MA5", "VolumeLag1"]]
y = data["Close"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

X_train.shape, X_test.shape

((704, 3), (177, 3))

## Step 4: Train Linear Regression Model

In [14]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R²:", r2)
print("Coefficients:", dict(zip(X.columns, model.coef_)))

RMSE: 42.007462286910894
R²: 0.9607413364219061
Coefficients: {'Lag1': np.float64(0.700745872756398), 'MA5': np.float64(0.29913845036174613), 'VolumeLag1': np.float64(-1.0764203262259482e-08)}


## Step 5: Plot Actual vs Predicted Prices

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(y_test.index, y_test, label="Actual")
plt.plot(y_test.index, y_pred, label="Predicted")
plt.legend()
plt.title("TVSMOTOR Stock Price Prediction — Linear Regression")
plt.xlabel("Date")
plt.ylabel("Closing Price (INR)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## ✅ Next Steps
- Try **Polynomial Regression** for comparison
- Add **technical indicators** like RSI, MACD, Bollinger Bands
- Compare with ML models like Random Forest or Gradient Boosting